<a href="https://colab.research.google.com/github/brookegrantham/firstRProject/blob/master/RoBERTa_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

reduced_set = pd.read_csv('reduced_set')
reduced_set = reduced_set[:30000]
reduced_set['label']= reduced_set['label'].replace({2:1})

In [2]:
len(reduced_set)

30000

In [3]:
from sklearn.model_selection import train_test_split

# Training = 0.7, validation = 0.1, test = 0.2
training_tweets, temp_tweets, training_labels, temp_labels = train_test_split(list(reduced_set['text']),list(reduced_set['label']), test_size=0.3, random_state=123)
val_tweets, test_tweets, val_labels, test_labels = train_test_split(temp_tweets, temp_labels, test_size=(1/3), random_state=123)

In [4]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def tokenize_function(dataset):
    model_inputs = tokenizer(dataset['text'], padding="max_length", truncation=True, max_length=100)
    return model_inputs

In [5]:
train = {'text': training_tweets, 'label': training_labels}
val = {'text': val_tweets, 'label': val_labels}
test = {'text': test_tweets, 'label': test_labels}

In [6]:
from datasets import Dataset

train_dataset = Dataset.from_dict(train)
val_dataset = Dataset.from_dict(val)
test_dataset = Dataset.from_dict(test)

In [7]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

  0%|          | 0/21 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="transformer_checkpoints",
    num_train_epochs=1,
)

In [9]:
from transformers import Trainer, RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained('roberta-base')

for param in model.roberta.parameters():
    param.requires_grad = False


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Step,Training Loss
500,0.693500
1000,0.692300
1500,0.689400
2000,0.687800
2500,0.685400


Saving model checkpoint to transformer_checkpoints/checkpoint-500
Configuration saved in transformer_checkpoints/checkpoint-500/config.json
Model weights saved in transformer_checkpoints/checkpoint-500/pytorch_model.bin
Saving model checkpoint to transformer_checkpoints/checkpoint-1000
Configuration saved in transformer_checkpoints/checkpoint-1000/config.json
Model weights saved in transformer_checkpoints/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to transformer_checkpoints/checkpoint-1500
Configuration saved in transformer_checkpoints/checkpoint-1500/config.json
Model weights saved in transformer_checkpoints/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to transformer_checkpoints/checkpoint-2000
Configuration saved in transformer_checkpoints/checkpoint-2000/config.json
Model weights saved in transformer_checkpoints/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to transformer_checkpoints/checkpoint-2500
Configuration saved in transformer_checkpoin

TrainOutput(global_step=2625, training_loss=0.6893207426525297, metrics={'train_runtime': 133.8345, 'train_samples_per_second': 156.91, 'train_steps_per_second': 19.614, 'total_flos': 1079166438000000.0, 'train_loss': 0.6893207426525297, 'epoch': 1.0})

In [10]:
import numpy as np
import torch

def predict_nn(trained_model, test_dataset):

    output = trained_model(attention_mask=torch.tensor(test_dataset["attention_mask"]).cuda(), input_ids=torch.tensor(test_dataset["input_ids"]).cuda())

    pred_labs = np.argmax(output["logits"].cpu().detach().numpy(), axis=1)

    gold_labs = test_dataset["label"]

    return gold_labs, pred_labs

gold_labs, pred_labs = predict_nn(model, test_dataset)

In [11]:
from sklearn.metrics import accuracy_score, f1_score
print("RoBERTa on the John Hopkins Twitter dataset:")
print(f'The accuracy score is {accuracy_score(pred_labs, gold_labs)}')
print(f'The f1-score is {f1_score(pred_labs,gold_labs)}')

RoBERTa on the John Hopkins Twitter dataset:
The accuracy score is 0.5526666666666666
The f1-score is 0.5785175879396984


In [12]:
x = {'True labels':gold_labs,'Predicted labels': pred_labs, 'text': test_tweets}

In [13]:
reduced_set

,Unnamed: 0,text,label
0,0,This week is going to be perfect~,1
1,1,"@jkpAhe6LeZk Gracias for the follow, ginger!",1
2,2,@e9nx7G79OxSjgUX but you're mental age is abou...,0
3,3,@mrpqvocKBHYI stil gota b there for our teamm...,1
4,4,About to watch this honey boo boo child show.....,1
...,...,...,...
29995,29995,It's really a peace sign that's all✌️ @ugtToRE...,1
29996,29996,@ft98ympaI I'm great! How about you?,0
29997,29997,AND I SAY HE'LL YEAH. HELL YEAH. FUCKIN RIGHT ...,0
29998,29998,Strawberries and Nutella hello,1
